### process uavsar multi segment sierra line
jack tarricone
nov 16th start

In [1]:
%matplotlib inline
import isce
import os
import glob
import numpy as np
import shutil
from osgeo import gdal
import matplotlib.pyplot as plt
import rasterio as rio
from rasterio.plot import show # plotting raster data
from rasterio.plot import show_hist #histograms of raster data
from uavsar_pytools.georeference import geolocate_uavsar

2023-11-29 09:42:39,861 - rasterio.session - DEBUG - Could not import boto3, continuing with reduced functionality.


/Users/jtarrico/miniconda/envs/isce2_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/jtarrico/miniconda/envs/isce2_env/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# get home dir
home = '/Users/jtarrico/'
os.chdir(home)
os.getcwd()

'/Users/jtarrico'

In [3]:
# working folder name
isce_directory = 'sierra_isce_multi'
    
# path to folder
working_dir = os.path.join(home,isce_directory)

In [4]:
# make new folder
# os.mkdir(working_dir)
print("Directory '%s' created" %isce_directory)

Directory 'sierra_isce_multi' created


In [5]:
# nativate to new folder
os.chdir(working_dir)
os.getcwd()

'/Users/jtarrico/sierra_isce_multi'

In [ ]:
## create subdirectories

# download
dl = 'download'
download_path = os.path.join(working_dir,dl)
# os.mkdir(download_path)

In [ ]:
# check folders were created
for file in os.listdir(working_dir):
    d = os.path.join(working_dir, file)
    if os.path.isdir(d):
        print(d)

# nativate to download folder
os.chdir(download_path)
os.getcwd()

## download

In [ ]:
# create .sh file from links

links = ['wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_01_BC.dop',
'wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_01_BC_s1_1x1.llh',
'wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_01_BC_s1_1x1.lkv',
'wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_01_BC_s2_1x1.llh',
'wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_01_BC_s2_1x1.lkv',
'wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_20002_001_200131_L090VV_01_BC.ann',
'wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_20006_002_200212_L090VV_01_BC.ann',
'wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_20009_023_200219_L090VV_01_BC.ann',
'wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_20016_005_200311_L090VV_01_BC.ann',
'wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_20014_000_200226_L090VV_01_BC.ann',
'wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_20002_001_200131_L090VV_01_BC_s1_1x1.slc',
'wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_20006_002_200212_L090VV_01_BC_s1_1x1.slc',
'wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_20009_023_200219_L090VV_01_BC_s1_1x1.slc',
'wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_20014_000_200226_L090VV_01_BC_s1_1x1.slc',
'wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_20016_005_200311_L090VV_01_BC_s1_1x1.slc',
'wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_20002_001_200131_L090VV_01_BC_s2_1x1.slc',
'wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_20006_002_200212_L090VV_01_BC_s2_1x1.slc',
'wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_20009_023_200219_L090VV_01_BC_s2_1x1.slc',
'wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_20014_000_200226_L090VV_01_BC_s2_1x1.slc',
'wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_20016_005_200311_L090VV_01_BC_s2_1x1.slc']

with open('downloads.sh', 'w') as f:
    for links in links:
        f.write(links)
        f.write('\n')

In [9]:
# list files downloaded
for i in os.listdir(download_path):
    file = os.path.join(download_path, i)
    print(file)

NameError: name 'download_path' is not defined

## merge segments

add these scripts to the stripmapp folder

```pair.py``` \
```group_segments_dayhrmin_4isce.py``` \
```make_ifg_vrt_xml_isce.py``` \
```insar.py``` \
```make_slc_vrt_xml_isce.py``` \
```cat_and_interfere_py3.py``` \
```unpackFrame_UAVSAR_JPLcode.py``` 

In [25]:
# nativate to new folder
os.chdir('./download')
os.getcwd()

'/Users/jtarrico/sierra_isce_multi/download'

In [26]:
# read segments
! group_segments_dayhrmin_4isce.py . > images.json

In [27]:
# generate ancillary files
! make_slc_vrt_xml_isce.py ./

8874 66664
8874 63032
unpackFrame_UAVSAR_JPLcode.py -i sierra_17305_20002_001_200131_L090VV_01_BC.ann -d ./sierra_17305_01_BC.dop -l 129696 -o ./20200131T1903
2023-11-28 16:31:23,612 - isce.Sensor.UAVSAR_Stack - INFO - extractDoppler: rho0, drho, prf = 11558.5978, 1.66551366, 445.46146464548985
2023-11-28 16:31:23,634 - isce.Sensor.UAVSAR_Stack - INFO - coeffs = [-1.18618649e+00  7.59851272e-04 -2.52651503e-07  5.67221397e-11
 -7.83341009e-15  5.86456631e-19 -1.80564378e-23]
2023-11-28 16:31:23,635 - isce.Sensor.UAVSAR_Stack - INFO - rms residual = [0.00114148]
2023-11-28 16:31:23,702 - isce.Sensor.UAVSAR_Stack - INFO - UAVSAR_Stack.extractDoppler: self.dopplerVals = {'Near': -1.1861864913881197}
2023-11-28 16:31:23,703 - isce.Sensor.UAVSAR_Stack - INFO - UAVSAR_Stack.extractDoppler: prf = 445.46146464548985
2023-11-28 16:31:23,705 - isce.Sensor.UAVSAR_Stack - INFO - UAVSAR_Stack: peg radius of curvature = 6361990.437831623
2023-11-28 16:31:23,706 - isce.Sensor.UAVSAR_Stack - INFO - UA

In [28]:
# create pairs .txt
! pair.py -n 1 < images.json > pairs.txt

In [29]:
# form insar pairs at 8x2
! cat_and_interfere_py3.py -d int -r 2 -a 8 images.json pairs.txt

Skipping 20200131T1903_20200212T2211 because it already exists.
Skipping 20200212T2211_20200219T2208 because it already exists.
Skipping 20200219T2208_20200226T2253 because it already exists.
Skipping 20200226T2253_20200311T1852 because it already exists.


In [30]:
# gen more anc files
! make_ifg_vrt_xml_isce.py -i int/

API open (R): int/20200212T2211_20200219T2208/20200212T2211_20200219T2208.int
API close:  int/20200212T2211_20200219T2208/20200212T2211_20200219T2208.int
API open (R): int/20200212T2211_20200219T2208/20200212T2211_20200219T2208.amp
API close:  int/20200212T2211_20200219T2208/20200212T2211_20200219T2208.amp
API open (R): int/20200212T2211_20200219T2208/20200212T2211_20200219T2208.cor
API close:  int/20200212T2211_20200219T2208/20200212T2211_20200219T2208.cor
API open (R): int/20200219T2208_20200226T2253/20200219T2208_20200226T2253.int
API close:  int/20200219T2208_20200226T2253/20200219T2208_20200226T2253.int
API open (R): int/20200219T2208_20200226T2253/20200219T2208_20200226T2253.amp
API close:  int/20200219T2208_20200226T2253/20200219T2208_20200226T2253.amp
API open (R): int/20200219T2208_20200226T2253/20200219T2208_20200226T2253.cor
API close:  int/20200219T2208_20200226T2253/20200219T2208_20200226T2253.cor
API open (R): int/20200131T1903_20200212T2211/20200131T1903_20200212T2211.in

## test plot

In [ ]:
w_int = './int/20200131T1903_20200212T2211/20200131T1903_20200212T2211.int.vrt'
coh_path = './int/20200131T1903_20200212T2211/20200131T1903_20200212T2211.cor.vrt'

# read int
ds = gdal.Open(w_int, gdal.GA_ReadOnly)
igram = ds.GetRasterBand(1).ReadAsArray()
ds = None

# read
coh_ds = gdal.Open(coh_path, gdal.GA_ReadOnly)
coh = coh_ds.GetRasterBand(1).ReadAsArray()
coh_ds = None

fig = plt.figure(figsize=(18, 14))

ax = fig.add_subplot(1,3,1)
ax.imshow(np.abs(igram), vmin = 0, vmax = .3, cmap = 'gray')
ax.set_title("Magnitude")
ax.set_axis_off()

ax = fig.add_subplot(1,3,2)
ax.imshow(np.angle(igram), cmap = 'jet', vmax = 3.14, vmin = -3.14)
ax.set_title("Phase")
ax.set_axis_off()

ax = fig.add_subplot(1,3,3)
ax.imshow(coh, vmin = 0 , vmax = 1, cmap = 'gray')
ax.set_title("Coherence")
ax.set_axis_off()

# Phase Unwrapping

In [36]:
# start in downloads directory
os.chdir('/Users/jtarrico/sierra_isce_multi')

In [31]:
# create text file of date list
! ls --color=never int/ >> datelist.txt

In [37]:
# print contents of that file
with open('datelist.txt') as f:
    lines = f.readlines()
    print(lines)

['20200212T2211_20200219T2208\n', '20200219T2208_20200226T2253\n', '20200226T2253_20200311T1852\n']


In [38]:
! unwrap.py -h

usage: unwrap.py [-h] -i INTFILE -u UNWPREFIX -c COHFILE [--nomcf]
                 [-a AZLOOKS] [-r RGLOOKS] [-d DEFOMAX] [-s REFERENCE]
                 [-m METHOD]

Unwrap interferogram using snaphu

optional arguments:
  -h, --help            show this help message and exit
  -i INTFILE, --ifg INTFILE
                        Input interferogram
  -u UNWPREFIX, --unwprefix UNWPREFIX
                        Output unwrapped file prefix
  -c COHFILE, --coh COHFILE
                        Coherence file
  --nomcf               Run full snaphu and not in MCF mode
  -a AZLOOKS, --alks AZLOOKS
                        Number of azimuth looks
  -r RGLOOKS, --rlks RGLOOKS
                        Number of range looks
  -d DEFOMAX, --defomax DEFOMAX
                        Max cycles of deformation
  -s REFERENCE, --reference REFERENCE
                        Reference directory
  -m METHOD, --method METHOD
                        unwrapping method


In [40]:
# make shell file to run unwrap
! awk -F "_" '{print "unwrap.py -i int/"$1"_"$2"/"$1"_"$2".int -u int/"$1"_"$2"/"$1"_"$2".unw -c int/"$1"_"$2"/"$1"_"$2".coh -a 8 -r 2 -s download/"$1"/ -m snaphu"}' datelist.txt >> unwall.sh

In [42]:
! sh unwall.sh

snaphu
int/20200212T2211_20200219T2208/referenceShelve/data
unwrapping method :  snaphu

snaphu v1.4.2
Reading wrapped phase from file int/20200212T2211_20200219T2208/20200212T2211_20200219T2208.int
No weight file specified.  Assuming uniform weights
Reading correlation data from file int/20200212T2211_20200219T2208/20200212T2211_20200219T2208.coh
Calculating smooth-solution cost parameters
Initializing flows with MCF algorithm
Integrating phase
Writing output to file int/20200212T2211_20200219T2208/20200212T2211_20200219T2208.unw_snaphu.unw
Growing connected components on second pass

snaphu v1.4.2
Reading unwrapped phase from file int/20200212T2211_20200219T2208/20200212T2211_20200219T2208.unw_snaphu.unw
No weight file specified.  Assuming uniform weights
Reading correlation data from file int/20200212T2211_20200219T2208/20200212T2211_20200219T2208.coh
Calculating smooth-solution cost parameters
Growing connected component mask
Writing connected components to file int/20200212T2211_2

In [2]:
# start in downloads directory
os.chdir('/Users/jtarrico/sierra_isce_multi/')

In [3]:
# list files generated in unwrapping process
!ls int/*/*

int/20200131T1903_20200212T2211/20200131T1903_20200212T2211.amp
int/20200131T1903_20200212T2211/20200131T1903_20200212T2211.amp.vrt
int/20200131T1903_20200212T2211/20200131T1903_20200212T2211.amp.xml
int/20200131T1903_20200212T2211/20200131T1903_20200212T2211.coh
int/20200131T1903_20200212T2211/20200131T1903_20200212T2211.coh.vrt
int/20200131T1903_20200212T2211/20200131T1903_20200212T2211.coh.xml
int/20200131T1903_20200212T2211/20200131T1903_20200212T2211.int
int/20200131T1903_20200212T2211/20200131T1903_20200212T2211.int.vrt
int/20200131T1903_20200212T2211/20200131T1903_20200212T2211.int.xml
int/20200131T1903_20200212T2211/20200131T1903_20200212T2211.unw_snaphu.unw
int/20200131T1903_20200212T2211/20200131T1903_20200212T2211.unw_snaphu.unw.conncomp
int/20200131T1903_20200212T2211/20200131T1903_20200212T2211.unw_snaphu.unw.conncomp.vrt
int/20200131T1903_20200212T2211/20200131T1903_20200212T2211.unw_snaphu.unw.conncomp.xml
int/20200131T1903_20200212T2211/20200131T1903_20200212T2211.unw_s

In [ ]:
# plot

# reading the multi-looked wrapped interferogram
ds = gdal.Open("int/20200212T2211_20200219T2208/20200212T2211_20200219T2208.int", gdal.GA_ReadOnly)
igram = ds.GetRasterBand(1).ReadAsArray()
ds = None

# reading the multi-looked unwrapped interferogram
ds = gdal.Open("int/20200212T2211_20200219T2208/20200212T2211_20200219T2208.unw_snaphu.unw", gdal.GA_ReadOnly)
igram_unw = ds.GetRasterBand(2).ReadAsArray()
ds = None

# reading the connected component file
ds = gdal.Open("int/20200212T2211_20200219T2208/20200212T2211_20200219T2208.unw_snaphu.unw.conncomp", gdal.GA_ReadOnly)
connected_components = ds.GetRasterBand(1).ReadAsArray()
ds = None

fig = plt.figure(figsize=(18, 16))

ax = fig.add_subplot(1,3,1)
cax=ax.imshow(np.angle(igram), cmap='jet')
ax.set_title("wrapped")
#ax.set_axis_off()
cbar = fig.colorbar(cax, ticks=[-3.14,0,3.14],orientation='horizontal')
cbar.ax.set_xticklabels(["$-\pi$",0,"$\pi$"])

ax = fig.add_subplot(1,3,2)
cax = ax.imshow(igram_unw, vmin = 2, vmax = 11, cmap = 'jet')
ax.set_title("unwrapped")
ax.set_axis_off()
cbar = fig.colorbar(cax, ticks=[2,0,11], orientation='horizontal')


ax = fig.add_subplot(1,3,3)
cax = ax.imshow(connected_components, cmap = 'jet')
ax.set_title("components")
ax.set_axis_off()
cbar = fig.colorbar(cax, ticks=[0,21] , orientation='horizontal')
cbar.ax.set_xticklabels([0,21])


connected_components = None

In [ ]:
! gdalinfo int_orginal/20200131T1903_20200212T2211/20200131T1903_20200212T2211.unw_snaphu.unw

## geocode

In [ ]:
from pathlib import Path
from glob import glob
import os
from uavsar_pytools.convert.tiff_conversion import read_annotation, array_to_tiff
from os.path import join, basename, dirname
from rasterio.vrt import WarpedVRT

# combine llh files
def combo_llhs(data_dir: Path, ann_fp: Path):
    """
    Combines segment .llh files into a single combined .llh file and saves as a binary with .vrt.
    """
    assert data_dir.exists()

    re_llhs = {'lat':[], 'lon': [], 'height':[]}
    for llh in sorted(data_dir.glob('*.llh')):
        segment = llh.stem.split('_')[-2].replace('s','')

        data = np.fromfile(llh, np.dtype('<f'))
        lat, lon, height = data[::3], data[1::3], data[2::3]
        for key, da in zip(re_llhs.keys(), [lat, lon, height]):
            re_llhs[key].extend(da)
                               
    full = np.empty(len(re_llhs['lat'])*3, dtype='>f')
    full[0::3] = re_llhs['lat']
    full[1::3] = re_llhs['lon']
    full[2::3] = re_llhs['height']

    # read ann file
    desc = read_annotation(ann_fp)
    print('reading .ann file')

    # read in number of rows from each llh file
    nrows1 = desc[f'llh_1_2x8.set_rows']['value']
    nrows2 = desc[f'llh_2_2x8.set_rows']['value']

    # add rows for new reshaping number
    nrows_new = nrows1 + nrows2
    print('new number of rows = ',nrows_new)
    
    # read in cols, same fo rboth
    ncols = desc[f'llh_1_2x8.set_cols']['value']
    dt = np.dtype('<f')

    # create empty arrays for lat and lon
    lat_array = np.empty((nrows_new, ncols))
    lon_array = np.empty((nrows_new, ncols))
    
    # fill each layer
    lat_array = full[::3].reshape(nrows_new, ncols)
    lon_array = full[1::3].reshape(nrows_new, ncols)

    # define path to bin file
    lat_output_file = data_dir / "multi_seg.lat"
    lon_output_file = data_dir / "multi_seg.lon"

    #### Save the array to bin file
    # lat
    lat_array.tofile(lat_output_file)
    print('.lat saved')
    # lon
    lon_array.tofile(lon_output_file)
    print('.lon saved')

    profile = {
    'driver': 'GTiff',
    'interleave': 'band',
    'tiled': False,
    'nodata': 0,
    'width': ncols,
    'height':nrows_new,
    'count':1,
    'dtype':'float32'
    }
    
    ### Save out tifs to be converted to vrt
    # lat
    with rio.open(join(str(lat_output_file) + '.tif'), 'w', **profile) as dst:
                dst.write(lat_array.astype(lat_array.dtype), 1)

    with rio.open(join(str(lon_output_file) + '.tif'), 'w', **profile) as dst:
                dst.write(lon_array.astype(lon_array.dtype), 1)

    # Add VRT file for each tif
    tifs = glob(join(data_dir, '*.tif')) # list all .llh files
    for tiff in tifs: # loop to open and translate .llh to .vrt, and save .vrt using gdal
        raster_dataset = gdal.Open(tiff, gdal.GA_ReadOnly) # read in rasters
        raster = gdal.Translate(join(data_dir, basename(tiff).replace('.tif','.vrt')), raster_dataset, format = 'VRT', outputType = gdal.GDT_Float32)
    raster_dataset = None

    print('new .lat and .lon with .vrt have saved')


In [ ]:
dir = Path('./sierra_multi_seg_llh_zach')
ann = Path('./sierra_multi_seg_llh_zach/sierra_17305_20002_001_200131_L090VV_01_BC.ann')

In [ ]:
full = combo_llhs(data_dir = dir, ann_fp = ann)

In [ ]:
# path to your unw.vrt
in_fp = 'int_orginal/20200131T1903_20200212T2211/20200131T1903_20200212T2211.unw_snaphu.unw'

# path to annotation file
lat_fp = '/Users/jtarrico/sierra_isce_multi/sierra_multi_seg_llh_zach/multi_seg.lat.vrt'

# path to annotation file
lon_fp = '/Users/jtarrico/sierra_isce_multi/sierra_multi_seg_llh_zach/multi_seg.lon.vrt'

# where the .tif is being save
out_fp = '/Users/jtarrico/sierra_isce_multi/sierra_multi_seg_llh_zach/0200131T1903_20200212T2211.unw_snaphu.unw.tif'

In [ ]:
from osgeo import gdal, osr

def geocodeUsingGdalWarp_multi(infile, latfile, lonfile, outfile,
                         insrs=4326, outsrs=None,
                         spacing=None, fmt='GTiff', bounds=None,
                         method='near'):
    '''
    From: Dr. Gareth Funning, UC Riverside, UNAVCO InSAR Short Course
    Geocode a swath file using corresponding lat, lon files
    '''
    sourcexmltmpl = '''    <SimpleSource>
      <SourceFilename>{0}</SourceFilename>
      <SourceBand>{1}</SourceBand>
    </SimpleSource>'''
    
    driver = gdal.GetDriverByName('VRT')
    tempvrtname = 'temp_ele.vrt'
    ds = gdal.OpenShared(infile, gdal.GA_ReadOnly)
    inds1 = ds.GetRasterBand(1).ReadAsArray()
    tempds = driver.Create(tempvrtname, inds1.shape[1], inds1.shape[0], 0)
    
    #for ii in range(inds.RasterCount):
    band = ds.GetRasterBand(1)
    tempds.AddBand(band.DataType)
    tempds.GetRasterBand(1).SetMetadata({'source_0': sourcexmltmpl.format(infile, 1)}, 'vrt_sources')
  
    sref = osr.SpatialReference()
    sref.ImportFromEPSG(insrs)
    srswkt = sref.ExportToWkt()
    tempds.SetMetadata({'SRS' : srswkt,
                        'X_DATASET': lonfile,
                        'X_BAND' : '1',
                        'Y_DATASET': latfile,
                        'Y_BAND' : '1',
                        'PIXEL_OFFSET' : '0',
                        'LINE_OFFSET' : '0',
                        'PIXEL_STEP' : '1',
                        'LINE_STEP' : '1'}, 
                        'GEOLOCATION')
    
    band = None
    tempds = None 
    inds = None
    
    if spacing is None:
        spacing = [None, None]
    warpOptions = gdal.WarpOptions(format=fmt,
                                xRes=spacing[0], yRes=spacing[0],
                                dstSRS=outsrs, outputBounds = bounds, dstNodata = -9999,
                                resampleAlg=method, geoloc=True)
    gdal.Warp(outfile, tempvrtname, options=warpOptions)
    os.remove('temp_ele.vrt')

In [ ]:
geocodeUsingGdalWarp(infile = in_fp,
                     latfile = lat_fp, 
                     lonfile = lon_fp, 
                     outfile = out_fp,
                     insrs=4326, outsrs=None,spacing=[.00005556,.00005556], fmt='GTiff', bounds=None,method='near')

In [ ]:
# path to your unw.vrt
in_fp = 'int_orginal/20200131T1903_20200212T2211/20200131T1903_20200212T2211.unw_snaphu.unw.conncomp'

# path to annotation file
lat_fp = '/Users/jtarrico/sierra_isce_multi/sierra_multi_seg_llh_zach/multi_seg.lat.vrt'

# path to annotation file
lon_fp = '/Users/jtarrico/sierra_isce_multi/sierra_multi_seg_llh_zach/multi_seg.lon.vrt'

# where the .tif is being save
out_fp = '/Users/jtarrico/sierra_isce_multi/sierra_multi_seg_llh_zach/0200131T1903_20200212T2211.unw_snaphu.unw.conncomp.tif'

In [ ]:
geocodeUsingGdalWarp(infile = in_fp,
                     latfile = lat_fp, 
                     lonfile = lon_fp, 
                     outfile = out_fp,
                     insrs=4326, outsrs=None,spacing=[.00005556,.00005556], fmt='GTiff', bounds=None,method='near')